In [1]:
!pip install -q transformers datasets scikit-learn accelerate

# SETUP

In [2]:
import torch
import pandas as pd
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments
)

from sklearn.metrics import f1_score

In [3]:
torch.cuda.is_available()

False

# Dataset loading

In [4]:
df = pd.read_csv("Trial_Data.csv")
df.head()

,text,lang,id,polarization,political,racial/ethnic,religious,gender/sexual,other,stereotype,vilification,dehumanization,extreme_language,lack_of_empathy,invalidation
0,እንደምን አደራችሁ ቤዛ! የ እለቱን ቃል በ ፓስተር ፍቅሬ በላይ እነሆ! ...,amh,amh_4a249cbbca6389d5120fe9b556f5ebf2,0,0,0,0,0,0,0,0,0,0,0,0
1,@USER ተሌግራምን እንደ አማራጭ መጠቀም መልካም ይመስለኛል::,amh,amh_01b5780721c676222b91ec97a797a066,0,0,0,0,0,0,0,0,0,0,0,0
2,አንደኛው አመት የፕሪቶርያው ስምምነት ምክንያት በማድረግ ከህዝባዊ ወያነ ...,amh,amh_7aabf3f2c41cbd032e6dbf56eccca9d7,0,0,0,0,0,0,0,0,0,0,0,0
3,ግደል እንዳልልህ.ሃጢያት ነው አውቃለሁ አትግደል አልልህ.ጨካኝ ፍጡር ናቸ...,amh,amh_16df45b84088cd0d09ca4fa3a0e5f35c,0,0,0,0,0,0,0,0,0,0,0,0
4,አለ ነገር አለ ነገር እንደው አለው ነገር እንዲህ ታመን ታመን አንገት ደ...,amh,amh_a28c14c4242c9387bc7a028cfad19c9f,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
TASK1_LABEL = "Polarization"

TASK2_LABELS = [
    "Political", "Racial/Ethnic", "Religious",
    "Gender/Sexual", "Other"
]

TASK3_LABELS = [
    "Stereotype", "Vilification", "Dehumanization", "Extreme Language",
    "Lack of Empathy", "Invalidation"
]

# Preprocessing

In [7]:
MODEL_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]